---
title: "Generate credit card transactions data and send to kafka topic"
date: 2021-04-25
type: technical_note
draft: false
---

# Generate credit card transactions data and send to kafka topic.

### Inspiration of this example was taken from [here](https://github.com/aws-samples/amazon-sagemaker-feature-store-streaming-aggregation).

#### Prerequisites 

In [1]:
#!pip install Faker

#### Imports 

In [2]:
from collections import defaultdict
from faker import Faker
import pandas as pd
import numpy as np
import datetime
import hashlib
import random
import math
import os

from hops import hdfs
from hops import pandas_helper as pandas

In [3]:
# Seed for Reproducibility
faker = Faker()
faker.seed_locale('en_US', 0)

In [4]:
SEED = 123
random.seed(SEED)
np.random.seed(SEED)
faker.seed_instance(SEED)

#### Constants 

In [5]:
TOTAL_UNIQUE_TRANSACTIONS = 27000
TOTAL_UNIQUE_USERS = 500
START_DATE = '2021-04-01 00:00:00'
END_DATE = '2021-04-04 00:01:00'
DATE_FORMAT = '%Y-%m-%d %H:%M:%S'

# change this according to your settings
KAFKA_BROKER_ADDRES = "broker.kafka.service.consul:9091"
KAFKA_TOPIC_NAME = "credit_card_transactions"

### Generate Transactions

#### Generate Unique Credit Card Numbers 
<p> Credit card numbers are uniquely assigned to users. Since, there are 10K users, we would want to generate 10K unique card numbers.</p>

In [6]:
def generate_unique_credit_card_numbers(n: int) -> list:
    cc_ids = set()
    for _ in range(n):
        cc_id = faker.credit_card_number(card_type='visa')
        cc_ids.add(cc_id)
    return list(cc_ids) 

In [7]:
credit_card_numbers = generate_unique_credit_card_numbers(TOTAL_UNIQUE_USERS)

In [8]:
assert len(credit_card_numbers) == TOTAL_UNIQUE_USERS 
assert len(credit_card_numbers[0]) == 16 # validate if generated number is 16-digit

In [9]:
# inspect random sample of credit card numbers 
random.sample(credit_card_numbers, 5)

['4048461056695943',
 '4644379952255281',
 '4859223670509563',
 '4894910969461020',
 '4114554907122106']

#### Generate Time Series


In [10]:
def generate_timestamps(n: int) -> list:
    start = datetime.datetime.strptime(START_DATE, DATE_FORMAT)
    end = datetime.datetime.strptime(END_DATE, DATE_FORMAT)
    timestamps = list()
    for _ in range(n):
        timestamp = faker.date_time_between(start_date=start, end_date=end, tzinfo=None).strftime(DATE_FORMAT)
        timestamps.append(timestamp)
    timestamps = sorted(timestamps)
    return timestamps

In [11]:
timestamps = generate_timestamps(TOTAL_UNIQUE_TRANSACTIONS)

In [12]:
assert len(timestamps) == TOTAL_UNIQUE_TRANSACTIONS

In [13]:
# inspect random sample of timestamps
random.sample(timestamps, 5)

['2021-04-01 23:19:53',
 '2021-04-01 09:24:45',
 '2021-04-01 03:15:07',
 '2021-04-02 09:06:25',
 '2021-04-02 23:01:18']

#### Generate Random Transaction Amounts 
<p>The transaction amounts are presumed to follow Pareto distribution, as it is logical for consumers to make many more smaller purchases than large ones. The break down of the distribution is shown in the table below.</p>


| Percentage        | Range (Amount in $)     |
| :-------------: | :----------: |
|  5\% | 0.01 to 1    |
| 7.5\%   | 1 to 10 |
| 52.5\%   | 10 to 100 |
| 25\%   | 100 to 1000 |
| 10\%   | 1000 to 10000 |

In [14]:
def get_random_transaction_amount(start: float, end: float) -> float:
    amt = round(np.random.uniform(start, end), 2)
    return amt

In [15]:
distribution_percentages = {0.05: (0.01, 1.01), 
                            0.075: (1, 11.01),
                            0.525: (10, 100.01),
                            0.25: (100, 1000.01),
                            0.10: (1000, 10000.01)}

In [16]:
amounts = []

for percentage, span in distribution_percentages.items():
    n = int(TOTAL_UNIQUE_TRANSACTIONS * percentage)
    start, end = span
    for _ in range(n):
        amounts.append(get_random_transaction_amount(start, end+1))
        
random.shuffle(amounts)

In [17]:
assert len(amounts) == TOTAL_UNIQUE_TRANSACTIONS

In [18]:
# inspect random sample of transaction amounts
random.sample(amounts, 5)

[53.86, 193.34, 949.23, 562.63, 631.5]

#### Generate Credit Card Transactions
<br>
<div style="text-align: justify">
Using the random credit card numbers, timestamps and transaction amounts generated in the above steps, 
we can generate random credit card transactions by combining them. The transaction id for the transaction is the md5
hash of the above mentioned entities.
</div>

In [19]:
def generate_transaction_id(timestamp: str, credit_card_number: str, transaction_amount: float) -> str:
    hashable = f'{timestamp}{credit_card_number}{transaction_amount}'
    hexdigest = hashlib.md5(hashable.encode('utf-8')).hexdigest()
    return hexdigest

In [20]:
transactions = []
for timestamp, amount in zip(timestamps, amounts):
    credit_card_number = random.choice(credit_card_numbers)
    transaction_id = generate_transaction_id(timestamp, credit_card_number, amount)
    transactions.append({'tid': transaction_id, 
                         'datetime': timestamp, 
                         'cc_num': credit_card_number, 
                         'amount': amount, 
                         'fraud_label': 0})

In [21]:
assert len(transactions) == TOTAL_UNIQUE_TRANSACTIONS

In [22]:
# inspect random sample of credit card transactions
random.sample(transactions, 1)

[{'tid': '84b82cf376f3688504c0801117cae081',
  'datetime': '2021-04-01 20:19:08',
  'cc_num': '4275154143939929',
  'amount': 5.23,
  'fraud_label': 0}]

In [23]:
FRAUD_RATIO = 0.0025 # percentage of transactions that are fraudulent
NUMBER_OF_FRAUDULENT_TRANSACTIONS = int(FRAUD_RATIO * TOTAL_UNIQUE_TRANSACTIONS)
ATTACK_CHAIN_LENGTHS = [3, 4, 5, 6, 7, 8, 9, 10]

#### Create Transaction Chains 

In [24]:
visited = set()
chains = defaultdict(list)

In [25]:
def size(chains: dict) -> int:
    counts = {key: len(values)+1 for (key, values) in chains.items()}
    return sum(counts.values())

In [26]:
def create_attack_chain(i: int):
    chain_length = random.choice(ATTACK_CHAIN_LENGTHS)
    for j in range(1, chain_length):
        if i+j not in visited:
            if size(chains) == NUMBER_OF_FRAUDULENT_TRANSACTIONS:
                break
            chains[i].append(i+j)
            visited.add(i+j)

In [27]:
while size(chains) < NUMBER_OF_FRAUDULENT_TRANSACTIONS:
    i = random.choice(range(TOTAL_UNIQUE_TRANSACTIONS))
    if i not in visited:
        create_attack_chain(i)
        visited.add(i)

In [28]:
assert size(chains) == NUMBER_OF_FRAUDULENT_TRANSACTIONS

#### Modify Transactions with Fraud Chain Attacks 

In [29]:
def generate_timestamps_for_fraud_attacks(timestamp: str, chain_length: int) -> list:
    timestamps = []
    timestamp = datetime.datetime.strptime(timestamp, DATE_FORMAT)
    for _ in range(chain_length):
        # interval in seconds between fraudulent attacks
        delta = random.randint(30, 120)
        current = timestamp + datetime.timedelta(seconds=delta)
        timestamps.append(current.strftime(DATE_FORMAT))
        timestamp = current
    return timestamps 

In [30]:
def generate_amounts_for_fraud_attacks(chain_length: int) -> list:
    amounts = []
    for percentage, span in distribution_percentages.items():
        n = math.ceil(chain_length * percentage)
        start, end = span
        for _ in range(n):
            amounts.append(get_random_transaction_amount(start, end+1))
    return amounts[:chain_length]

In [31]:
for key, chain in chains.items():
    transaction = transactions[key]
    timestamp = transaction['datetime']
    cc_num = transaction['cc_num']
    amount = transaction['amount']
    transaction['fraud_label'] = 1
    inject_timestamps = generate_timestamps_for_fraud_attacks(timestamp, len(chain))
    inject_amounts = generate_amounts_for_fraud_attacks(len(chain))
    random.shuffle(inject_amounts)
    for i, idx in enumerate(chain):
            original_transaction = transactions[idx]
            inject_timestamp = inject_timestamps[i]
            original_transaction['datetime'] = inject_timestamp
            original_transaction['fraud_label'] = 1
            original_transaction['cc_num'] = cc_num
            original_transaction['amount'] = inject_amounts[i]
            original_transaction['tid'] = generate_transaction_id(inject_timestamp, cc_num, amount)
            transactions[idx] = original_transaction

## Produce transactions

In [32]:
from hops import kafka
from hops import tls
from hops import hdfs
import json
from confluent_kafka import Producer

In [33]:
config = {
    "bootstrap.servers": KAFKA_BROKER_ADDRES,
    "security.protocol": kafka.get_security_protocol(),
    "ssl.ca.location": tls.get_ca_chain_location(),
    "ssl.certificate.location": tls.get_client_certificate_location(),
    "ssl.key.location": tls.get_client_key_location(),
    "group.id": "1"
}

producer = Producer(config)

In [34]:
transaction_labels = []

i = 0
for transaction in transactions:
    transaction_label = {}
    poped_label = transaction.pop("fraud_label")
    transaction_label["cc_num"]=transaction["cc_num"]
    transaction_label["fraud_label"]=poped_label
    transaction_labels.append(transaction_label)
    if i % 2000 == 0:
        print(json.dumps(transaction))
    producer.produce(KAFKA_TOPIC_NAME, json.dumps(transaction))
    producer.flush()
    i += 1

{"tid": "5380cc723c21f20b2408dcc1832b316f", "datetime": "2021-04-01 00:00:08", "cc_num": "4751637328229168", "amount": 71.28}
{"tid": "97117a3db0b5ee2f8dfa0208c5d4205c", "datetime": "2021-04-01 05:25:13", "cc_num": "4526611032294580", "amount": 76.63}
{"tid": "4c9eb202101e6a730daf908abdebd11b", "datetime": "2021-04-01 10:39:29", "cc_num": "4332149413304557", "amount": 361.98}
{"tid": "3692ab7ada3b64152457c412a6afe794", "datetime": "2021-04-01 16:00:23", "cc_num": "4226063306212844", "amount": 235.13}
{"tid": "69562cd10229e56dcede814a637c011c", "datetime": "2021-04-01 21:18:41", "cc_num": "4694511957134520", "amount": 58.63}
{"tid": "31d00449d2d415bfaab5d941865b5528", "datetime": "2021-04-02 02:31:04", "cc_num": "4834237757774643", "amount": 603.34}
{"tid": "b782c271bf19cdd0b0d2afd4a2fd15cd", "datetime": "2021-04-02 07:56:40", "cc_num": "4791850266678443", "amount": 3.6}
{"tid": "e974ab1d4179970ab972d2c9fdfe59a3", "datetime": "2021-04-02 13:16:07", "cc_num": "4987068065246544", "amount"

## Store labels in the Feature Store

In [37]:
import hsfs
connection = hsfs.connection()
# get a reference to the feature store, you can access also shared feature stores by providing the feature store name
fs = connection.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.


In [38]:
labels_df = pd.DataFrame.from_records(transaction_labels)
reduced_labels_df = labels_df.groupby(['cc_num'], as_index=False).agg('max')

fg = fs.get_feature_group("fraud_labels", version=1)
fg.insert(reduced_labels_df)

Configuring ingestion job...
Uploading Pandas dataframe...
Launching ingestion job...
Ingestion Job started successfully, you can follow the progress at https://hopsworks.glassfish.service.consul:8182/hopsworks/#!/project/120/jobs
